In [27]:
# ## 1. Suppress Warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [28]:
# ## 2. Import Required Libraries
import pandas as pd
import os
import scipy.stats as st
from fuzzywuzzy import process

In [29]:
# ## 3. Set Paths for Dataset Files
BASE = os.getcwd()
DATASETS_DIR = os.path.join(BASE, 'datasets')

PERF_DATASET_NAME = 'IPL_ball_by_ball_updated till 2024.csv'
PERF_DATASET_PATH = os.path.join(DATASETS_DIR, PERF_DATASET_NAME)

SALARY_DATASET_NAME = 'IPL SALARIES 2024.xlsx'
SALARY_DATASET_PATH = os.path.join(DATASETS_DIR, SALARY_DATASET_NAME)

In [30]:
# ## 4. Read IPL Performance Data
df = pd.read_csv(PERF_DATASET_PATH, low_memory=False)

In [31]:
df.head(3)  # Preview the first few rows

,Match id,Date,Season,Batting team,Bowling team,Innings No,Ball No,Bowler,Striker,Non Striker,runs_scored,extras,type of extras,score,score/wicket,wicket_confirmation,wicket_type,fielders_involved,Player Out
0,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.1,P Kumar,SC Ganguly,BB McCullum,0,1,legbyes,1,1/0,0,NaN,NaN,NaN
1,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.2,P Kumar,BB McCullum,SC Ganguly,0,0,NaN,1,1/0,0,NaN,NaN,NaN
2,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.2,P Kumar,BB McCullum,SC Ganguly,0,1,wides,2,2/0,0,NaN,NaN,NaN


In [32]:
df.columns  # Check column names

Index(['Match id', 'Date', 'Season', 'Batting team', 'Bowling team',
       'Innings No', 'Ball No', 'Bowler', 'Striker', 'Non Striker',
       'runs_scored', 'extras', 'type of extras', 'score', 'score/wicket',
       'wicket_confirmation', 'wicket_type', 'fielders_involved',
       'Player Out'],
      dtype='object')

In [33]:
df.info()  # Summary info about columns and types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255759 entries, 0 to 255758
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Match id             255759 non-null  int64  
 1   Date                 255759 non-null  object 
 2   Season               255759 non-null  object 
 3   Batting team         255759 non-null  object 
 4   Bowling team         255759 non-null  object 
 5   Innings No           255759 non-null  int64  
 6   Ball No              255759 non-null  float64
 7   Bowler               255759 non-null  object 
 8   Striker              255759 non-null  object 
 9   Non Striker          255759 non-null  object 
 10  runs_scored          255759 non-null  int64  
 11  extras               255759 non-null  int64  
 12  type of extras       13823 non-null   object 
 13  score                255759 non-null  int64  
 14  score/wicket         255759 non-null  object 
 15  wicket_confirmati

In [34]:
# ## 5. Drop Unnecessary Columns
df.drop([
    "Batting team", "Bowling team", "Ball No", "Non Striker", "extras", "score",
    "score/wicket", "type of extras", "wicket_type", "fielders_involved", "Player Out"
], axis=1, inplace=True)

In [35]:
# ## 6. Extract Year from Date Column
df["Year"] = pd.to_datetime(df["Date"], format="%d-%m-%Y").dt.year

In [36]:
# ## 7. Aggregate Runs and Wickets by Year, Innings, Player, Match
a1 = df.groupby(['Year', 'Innings No', 'Striker', 'Bowler']) \
        .agg({"runs_scored": "sum", "wicket_confirmation": "sum"}) \
        .reset_index()

In [37]:
runs = a1.groupby(['Year', 'Innings No', 'Striker']) \
         .agg({"runs_scored": "sum"}).reset_index()
wickets = a1.groupby(['Year', 'Innings No', 'Bowler']) \
            .agg({"wicket_confirmation": "sum"}).reset_index()

In [38]:
# ## 8. Identify Top 3 Players Each Year
seasons = runs["Year"].unique()
print(seasons)

[2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022 2023 2024]


In [39]:
for season in seasons:
    runs_season = runs[runs["Year"] == season]
    wickets_season = wickets[wickets["Year"] == season]

    print(f"Year: {season}\n")
    print("Top 3 Run Scorers:")
    print(runs_season.sort_values(by="runs_scored", ascending=False).head(3)[['Striker', 'runs_scored']])

    print("\nTop 3 Wicket Takers:")
    print(wickets_season.sort_values(by="wicket_confirmation", ascending=False).head(3)[['Bowler', 'wicket_confirmation']])
    print("\n" + "="*50 + "\n")

Year: 2008

Top 3 Run Scorers:
       Striker  runs_scored
31   G Gambhir          334
241  SR Watson          321
262  YK Pathan          318

Top 3 Wicket Takers:
           Bowler  wicket_confirmation
75  Sohail Tanvir                   15
82      VY Mahesh                   14
44    MF Maharoof                   14


Year: 2009

Top 3 Run Scorers:
            Striker  runs_scored
334       ML Hayden          363
376        SK Raina          352
278  AB de Villiers          343

Top 3 Wicket Takers:
        Bowler  wicket_confirmation
202  DP Nannes                   16
244   RP Singh                   15
181    A Nehra                   14


Year: 2010

Top 3 Run Scorers:
          Striker  runs_scored
644  SR Tendulkar          371
716     JH Kallis          368
646     SS Tiwary          342

Top 3 Wicket Takers:
              Bowler  wicket_confirmation
381         A Kumble                   16
411         DW Steyn                   16
518  Harbhajan Singh                   16



In [40]:
# ## 9. Prepare Data for Distribution Fitting
runs_per_year = df.groupby(['Year', 'Striker'])[['runs_scored']].sum().reset_index()
wickets_per_year = df.groupby(['Year', 'Bowler'])[['wicket_confirmation']].sum().reset_index()
runs_per_year.sort_values(['Year', 'runs_scored'], ascending=False, inplace=True)
wickets_per_year.sort_values(['Year', 'wicket_confirmation'], ascending=False, inplace=True)

In [41]:
last_3_seasons = runs_per_year['Year'].unique().tolist()[:3]

In [42]:
# ## 10. Dictionary for Top 3 Players (last 3 seasons)
top_3_dict = {}
for season in last_3_seasons:
    top_3_dict[season] = {
        "batsmen": {k: [] for k in runs_per_year[runs_per_year["Year"] == season]["Striker"].to_list()[:3]},
        "bowlers": {k: [] for k in wickets_per_year[wickets_per_year["Year"] == season]["Bowler"].to_list()[:3]}
    }

In [43]:
# ## 11. List of Candidate Distributions for Fitting
distribs = [
    "alpha", "beta", "betaprime", "burr12", "crystalball", "dgamma", "dweibull", "erlang",
    "exponnorm", "f", "fatiguelife", "gamma", "gengamma", "gumbel_l", "johnsonsb",
    "kappa4", "lognorm", "nct", "norm", "norminvgauss", "powernorm", "rice",
    "recipinvgauss", "t", "trapezoid", "truncnorm"
]

In [44]:
# ## 12. Function to Get Best Fit Distribution
def get_best_distrib(data):
    results = []
    params = {}
    for distrib in distribs:
        dist = getattr(st, distrib)
        param = dist.fit(data)
        params[distrib] = param
        _, p = st.kstest(data, distrib, args=param)
        results.append((distrib, p))

    best_distrib, best_p = max(results, key=lambda x: x[1])

    print(f"Best Fitting Distribution: {best_distrib}")
    print(f"Best P-Value: {best_p}")
    print(f"Params for Best Fit: {str(params[best_distrib])}")

    return [best_distrib, best_p, params[best_distrib]]

In [45]:
# ## 13. Fit Distributions for Top 3 Players Each Season
runs = df.groupby(['Year', 'Striker','Match id'])[['runs_scored']].sum().reset_index()
wickets = df.groupby(['Year', 'Bowler', 'Match id'])[['wicket_confirmation']].sum().reset_index()

for year in top_3_dict.keys():
    for striker in top_3_dict[year]["batsmen"].keys():
        print("************************\n")
        print(f"Year: {year} | Batsman: {striker}\n")
        top_3_dict[year]["batsmen"][striker] = get_best_distrib(runs[runs['Striker'] == striker]['runs_scored'])
        print('\n\n')

    for bowler in top_3_dict[year]["bowlers"].keys():
        print("************************\n")
        print(f"Year: {year} | Bowler: {striker}\n")
        top_3_dict[year]["bowlers"][bowler] = get_best_distrib(wickets[wickets['Bowler'] == bowler]['wicket_confirmation'])
        print('\n\n')

************************

Year: 2024 | Batsman: RD Gaikwad

Best Fitting Distribution: nct
Best P-Value: 0.5881570496217834
Params for Best Fit: (np.float64(5.718048022849898), np.float64(9.399490726283615), np.float64(-54.25277343780452), np.float64(8.497060689079994))



************************

Year: 2024 | Batsman: V Kohli

Best Fitting Distribution: beta
Best P-Value: 0.7807091136830002
Params for Best Fit: (np.float64(0.8162762768683594), np.float64(2.339175317753771), np.float64(-1.208864897132228e-31), np.float64(130.79369265890057))



************************

Year: 2024 | Batsman: B Sai Sudharsan

Best Fitting Distribution: f
Best P-Value: 0.9743698730235856
Params for Best Fit: (np.float64(7.230079510849502), np.float64(94.80990591947705), np.float64(-0.4687012931969091), np.float64(39.842021248481544))



************************

Year: 2024 | Bowler: B Sai Sudharsan

Best Fitting Distribution: alpha
Best P-Value: 0.0002993252328930708
Params for Best Fit: (np.float64(5.2

In [46]:
# ## 14. Assigned Player Distribution Fit (N Pooran)
my_player = "N Pooran"
my_data = runs[runs['Striker'] == my_player]['runs_scored']
get_best_distrib(my_data)

Best Fitting Distribution: dgamma
Best P-Value: 0.17382307012588938
Params for Best Fit: (np.float64(1.941154704943994), np.float64(22.522643564002294), np.float64(8.834432758239476))


['dgamma',
 np.float64(0.17382307012588938),
 (np.float64(1.941154704943994),
  np.float64(22.522643564002294),
  np.float64(8.834432758239476))]

In [47]:
# ## 15. Total Stats by Year
total_runs_per_year = df.groupby(['Year', 'Striker'])[['runs_scored']].sum().reset_index().sort_values(['Year', 'runs_scored'], ascending=False)
total_wickets_per_year = df.groupby(['Year', 'Bowler'])[['wicket_confirmation']].sum().reset_index().sort_values(['Year', 'wicket_confirmation'], ascending=False)

In [48]:
# ## 16. Load Salary Dataset
salary_df = pd.read_excel(SALARY_DATASET_PATH)

In [49]:
# ## 17. Filter 2024 Data
total_runs_2024 = total_runs_per_year[total_runs_per_year['Year'] == 2024]
total_wickets_2024 = total_wickets_per_year[total_wickets_per_year['Year'] == 2024]

In [50]:
# ## 18. Fuzzy Matching for Player Names
def match_names(name, names_list, threshold=80):
    if not name or not isinstance(name, str):
        return None

    result = process.extractOne(name, names_list)
    if result:
        match, score = result # type: ignore
        return match if score >= threshold else None
    return None

In [51]:
# ## 19. Correlation for Batsmen
df_striker_salary = salary_df.copy()
df_striker_salary['Matched Player'] = df_striker_salary['Player'].apply(lambda x: match_names(x, total_runs_2024['Striker'].tolist()))
df_striker_merged = pd.merge(df_striker_salary, total_runs_2024, left_on='Matched Player', right_on='Striker')

corr_striker = df_striker_merged['Rs'].corr(df_striker_merged['runs_scored'])
print('Correlation between Salary and Runs in 2024:', corr_striker)

Correlation between Salary and Runs in 2024: 0.3061248376582167


In [52]:
# ## 20. Correlation for Bowlers
df_bowler_salary = salary_df.copy()
df_bowler_salary['Matched Player'] = df_bowler_salary['Player'].apply(lambda x: match_names(x, total_wickets_2024['Bowler'].tolist()))
df_bowler_merged = pd.merge(df_bowler_salary, total_wickets_2024, left_on='Matched Player', right_on='Bowler')

corr_bowler = df_bowler_merged['Rs'].corr(df_bowler_merged['wicket_confirmation'])
print('Correlation between Salary and Wickets in 2024:', corr_bowler)

Correlation between Salary and Wickets in 2024: 0.056932579421469245
